In [32]:
import argparse
import logging
import matplotlib.pyplot as plt
from multiprocess import Pool
import numpy as np
import pandas as pd
import random
import sys
import operator
import math
import graphviz


from sklearn.datasets import load_boston, load_breast_cancer
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.linear_model import Ridge
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils._testing import assert_equal, assert_almost_equal
from sklearn.utils.validation import check_random_state

from gplearn.genetic import SymbolicClassifier, SymbolicRegressor
from gplearn.genetic import SymbolicTransformer
from gplearn.functions import make_function

# Graphs and dataset
import matplotlib.pyplot as plt
import seaborn as sb
from pmlb import fetch_data

# Target formula:
def target(x):
    return x**4 + x**3 + x**2 + x

x = np.arange(-1, 1, 1/10.)
y = target(x)

# SCOOP
#toolbox.register("map", futures.map)

# Best configurations as commandlines (first number is the configuration ID; same order as above):
# 45   --pop=438 --cros=0.53 --mut=0.13
# 104  --pop=438 --cros=0.53 --mut=0.19 based on irace

def main(POP, CXPB, MUTPB):

    rng = check_random_state(0)

    # Training samples
    X_train = rng.uniform(-1, 1, 100).reshape(50, 2)
    y_train = X_train[:, 1]**4 + X_train[:, 1]**3 + X_train[:, 1]**2 + X_train[:, 1]

    # Testing samples
    X_test = rng.uniform(-1, 1, 100).reshape(50, 2)
    y_test = X_test[:, 1]**4 + X_test[:, 1]**3 + X_test[:, 1]**2 + X_test[:, 1]
    
    # line =plt.plot(graph)
    # plt.show()

    # line =plt.plot(num_features)
    # plt.show()
    
    # To ensure CXPB + MUTPB are 1 or less than
    Prob = MUTPB + CXPB
    if Prob > 1:
        MUTPB = 1 - CXPB
    
    est_gp = SymbolicRegressor(population_size=POP,
                           generations=20, stopping_criteria=0.01,
                           p_crossover=CXPB, p_subtree_mutation=MUTPB,
                           p_hoist_mutation=0, p_point_mutation=0,
                           max_samples=0.9, verbose=1,
                           parsimony_coefficient=0.01, random_state=0)
                           
    est_gp.fit(X_train, y_train)
    idx = est_gp._program.parents['parent_idx']
    fade_nodes = est_gp._program.parents['parent_nodes']
    print(est_gp._programs[-2][idx])
    print('Fitness:', est_gp._programs[-2][idx].fitness_)
    dot_data = est_gp._programs[-2][idx].export_graphviz(fade_nodes=fade_nodes)
    graph = graphviz.Source(dot_data)
    graph.view()

if __name__ == "__main__":
    main(438,0.53,0.13)
    

    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0    37.42          411.403       15         0.388106         0.312618      6.05s
   1    11.73          1.88704       15         0.338316         0.760723      3.36s
   2     7.94          5.15614       11         0.228364         0.399213      3.06s
   3     5.91         0.832807       11         0.231597         0.370119      2.74s
   4     8.63          1.02651       17         0.154633         0.136715      2.67s
   5    11.20          1.20826       21         0.106583         0.042936      2.51s
   6     9.41         0.541011       25        0.0492796         0.101022      2.36s
   7     7.00         0.433194        7          0.12619         0.624483      2.18s
   8     7.00          3.28224        7         0.123291         0.650574  

IndentationError: expected an indented block (<ipython-input-26-685334f241bf>, line 11)